<a href="https://colab.research.google.com/github/hoho303/hoho303.github.io/blob/master/insect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from imutils import paths
from keras.constraints import max_norm
from keras.layers.normalization import BatchNormalization
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.optimizers import RMSprop,Adam
from keras.applications import VGG19,VGG16,Xception
from keras.regularizers import l2,l1
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras.layers.core import Flatten
import numpy as np
import random
import os
from matplotlib import pyplot

In [0]:
image_path = list(paths.list_images('./drive/My Drive/dataset/'))

random.shuffle(image_path)

In [0]:
labels = [p.split(os.path.sep)[-2] for p in image_path]

le = LabelEncoder()
labels = le.fit_transform(labels)

# One-hot encoding
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [12]:
list_image = []
for (j, imagePath) in enumerate(image_path):
    image = load_img(imagePath, target_size=(224, 224))
    image = img_to_array(image)
    
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    
    list_image.append(image)
    
list_image = np.vstack(list_image)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:914: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


In [13]:
# Load model VGG 19 của ImageNet dataset, include_top=False để bỏ phần Fully connected layer ở cuối.
baseModel = VGG19(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Xây thêm các layer
# Lấy output của ConvNet trong VGG16
fcHead = baseModel.output

# Flatten trước khi dùng FCs
fcHead = Flatten(name='flatten')(fcHead)

# Thêm FC
fcHead = Dense(256, activation='relu')(fcHead)
#fcHead = BatchNormalization()(fcHead)
fcHead = Dropout(0.5)(fcHead)
# Output layer với softmax activation
fcHead = Dense(15, activation='softmax')(fcHead)
# Xây dựng model bằng việc nối ConvNet của VGG19 và fcHead
model = model = Model(inputs=baseModel.input, outputs=fcHead)

Instructions for updating:
Colocations handled automatically by placer.
80142336/80134624 [==============================] - 1s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
X_train, X_test, y_train, y_test = train_test_split(list_image, labels, test_size=0.2, random_state=42)

In [0]:
aug_train = ImageDataGenerator(rescale = 1./255,
                                shear_range=0.2,
                                horizontal_flip = True,
                                fill_mode = "nearest",
                                zoom_range = 0.2,
                                width_shift_range = 0.1,
                                height_shift_range=0.1,
                                rotation_range=30)
# aug_train = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, 
#                          zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
# augementation cho test
aug_test= ImageDataGenerator(rescale = 1./255)

In [0]:
# freeze VGG model
for layer in baseModel.layers:
    layer.trainable = False
    
opt = Adam(lr=0.001)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
numOfEpoch = 30
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
184/184 [==============================] - 109s 592ms/step - loss: 2.8614 - acc: 0.1456 - val_loss: 2.2809 - val_acc: 0.2989
Epoch 2/30
102/184 [===============>..............] - ETA: 36s - loss: 2.4497 - acc: 0.1719

In [0]:
for layer in model.layers:
    layer.trainable = True

numOfEpoch = 100
opt = SGD(lr=0.001,decay=1e-6, momentum=0.9, nesterov=True)
model.compile(opt, 'categorical_crossentropy', ['accuracy'])
H = model.fit_generator(aug_train.flow(X_train, y_train, batch_size=32), 
                        steps_per_epoch=len(X_train)//32,
                        validation_data=(aug_test.flow(X_test, y_test, batch_size=32)),
                        validation_steps=len(X_test)//32,
                        epochs=numOfEpoch)

In [0]:
pyplot.plot(H.history['acc'], label='train')
pyplot.plot(H.history['val_acc'], label='test')
pyplot.legend()
pyplot.show()

In [0]:
pyplot.plot(H.history['loss'], label='train')
pyplot.plot(H.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()